In [1]:
import sys

In [8]:
from urllib.request import urlretrieve
import os

import itk
import imagej
import numpy as np



In [10]:
from itkwidgets import view

AttributeError: module 'itk' has no attribute 'image'

In [4]:
# Initialize imagej
ij = imagej.init('/Applications/Fiji.app')
print(ij.getVersion())

2.14.0/1.54f


In [5]:
file_name = 'General_EduRes_Heart_BloodVessels_0.jpg'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5afe74408d777f15ebe1d701/download'
    urlretrieve(url, file_name)

In [11]:
image = itk.imread(file_name, itk.ctype('float'))


AttributeError: module 'itk' has no attribute 'imread'

In [8]:
view(image)

NameError: name 'view' is not defined

In [ ]:
print(type(image))

image_arr = itk.array_view_from_image(image)
print(type(image_arr))

image_java = ij.py.to_java(image_arr)
print(type(image_java))

In [ ]:
# Invoke the Frangi vesselness op.
vessels = np.zeros(image_arr.shape, dtype=np.float32)
ij.op().filter().frangiVesselness(ij.py.to_java(vessels),
                                  image_java,
                                  [1, 1],
                                  20)

In [ ]:
view(vessels)

In [1]:
import imagej
import os



In [2]:
# Initialize ImageJ
ij = imagej.init('/Applications/Fiji.app')

[DEBUG] Creating service: org.scijava.event.DefaultEventService
[DEBUG] Creating service: org.scijava.log.StderrLogService
[DEBUG] Created service 'org.scijava.log.StderrLogService' in 0 ms
[DEBUG] Creating service: org.scijava.thread.DefaultThreadService
[DEBUG] Created service 'org.scijava.thread.DefaultThreadService' in 1 ms
[DEBUG] Created service 'org.scijava.event.DefaultEventService' in 34 ms
[DEBUG] Creating service: org.scijava.app.DefaultAppService
[DEBUG] Creating service: org.scijava.plugin.DefaultPluginService
[DEBUG] Found 2231 plugins.
[DEBUG] - class='net.imagej.plugins.tools.KonamiHandler', name='Konami', priority=Infinity, enabled=true, pluginType=Tool
[DEBUG] - class='org.scijava.convert.NullConverter', priority=1.0E300, enabled=true, pluginType=Converter
[DEBUG] - class='org.scijava.module.process.DebugPostprocessor', priority=1.0E300, enabled=true, pluginType=PostprocessorPlugin
[DEBUG] - class='org.scijava.module.process.DebugPostprocessor', priority=1.0E300, enab

[java.lang.Enum.toString] [DEBUG] publish(
	context = org.scijava.Context@e8bd694
	consumed = false
	items[0] = <jpype._jproxy.proxy.PythonScriptRunner object at 0x344a92990>,null,null), called from non-EDT Thread:null


In [ ]:

# Set the directory and file paths
dir_path = "/Users/vschastlivaia/Library/CloudStorage/OneDrive-IBEC/Documents/GitHub/lab-data-analysis/Nanoparticle characterization/"
pla_data_file = "25%AP2_PEGPDPA.txt"
cells_count_file = "cells_count.csv"

# Create specific folders for cells and PLA images
os.makedirs(os.path.join(dir_path, "cells_images"), exist_ok=True)
os.makedirs(os.path.join(dir_path, "PLA_images"), exist_ok=True)

# Open the PLA data file for writing
with open(os.path.join(dir_path, pla_data_file), "w") as f:
	f.write("Filename,Raw_PLA_signal,dx,dy,dz,Image_Volume_um3\n")

# Set the measurements
ij.run("Set Measurements...", "area mean integrated redirect=None decimal=3")

# Loop through the images
images = nImages
for i in range(images):
	title = getTitle()
	out = title.replace('/', ' - ')  # Sanitize input filename
	dx, dy, dz, um = getInfo("voxel.size").split(",")
	width, height, channels, slices, frames = getDimensions()

	# Split channels
	ij.run("Split Channels")

	# Cells counting
	ij.selectWindow('C2-' + title)
	ij.run("Z Project...", "projection=[Sum Slices]")
	ij.run("Convert to Mask")
	ij.run("Smooth")
	ij.run("Convert to Mask")
	ij.run("Erode")
	ij.run("Dilate")
	ij.run("Analyze Particles...", "size=10-Infinity circularity=0.00-0.90 show=Outlines clear summarize")
	ij.close('C2-' + title)
	ij.selectWindow('Drawing of SUM_C2-' + title)
	ij.rename("MAX")
	ij.selectWindow('SUM_C2-' + title)
	ij.rename("MASK")
	ij.run("Combine...", "stack1=MAX stack2=MASK")
	ij.saveAs("PNG", os.path.join(dir_path, "cells_images", out + "_count.png"))
	ij.close()
	ij.close('C2-' + title)

	# PLA analysis
	ij.selectWindow('C1-' + title)
	ij.run("Z Project...", "projection=[Sum Slices]")
	ij.selectWindow('SUM_C1-' + title)
	# Use the line below for bright datasets (for your samples)
	# ij.setAutoThreshold()
	# Use the line below for Negative Controls
	ij.setAutoThreshold("MaxEntropy dark")
	ij.run("Convert to Mask")
	ij.run("Measure")
	signal = ij.getResult("RawIntDen", 0)
	volume = width * height * slices * float(dx) * float(dy) * float(dz)
	with open(os.path.join(dir_path, pla_data_file), "a") as f:
		f.write(f"{title}.tif,{signal},{dx},{dy},{dz},{volume}\n")
	ij.run("Invert LUT")
	ij.rename("mask")
	ij.selectWindow('C1-' + title)
	ij.run("Z Project...", "projection=[Sum Slices]")
	ij.selectWindow('C1-' + title)
	ij.close()
	ij.selectWindow('SUM_C1-' + title)
	ij.rename("original")
	ij.run("8-bit")
	name1 = "SUM_C1-" + title + "-1"
	name2 = "SUM_C1-" + title
	ij.run("Combine...", "stack1=original stack2=mask")
	ij.saveAs("PNG", os.path.join(dir_path, "PLA_images", out + "_analysed.png"))
	ij.close()

# Saving cells counting as csv file
if ij.isOpen("Results"):
	ij.selectWindow("Results")
	ij.run("Close")
ij.selectWindow("Summary")
ij.saveAs("Results", os.path.join(dir_path, "cells_images", cells_count_file))
ij.close(cells_count_file)


##PLEASE TAKE IN MIND THAT IN THIS MACRO THE CHANNEL 2 REFERS TO THE NUCLEI STAINING!!!!!!!!!!!!!

images = nImages;
dir=getInfo("image.directory");

##create specific folders for cells and PLA images
File.makeDirectory(dir + "cells_images");
File.makeDirectory(dir + "PLA_images");

f = File.open(dir + "PLA_data.txt");
f = File.append("Filename,Raw_PLA_signal,dx,dy,dz,Image_Volume_um3", dir + "PLA_data.txt");
run("Set Measurements...", "area mean integrated redirect=None decimal=3");

for (i=0; i<images; i++) {
	title = getTitle();
	out = replace(title, '/', ' - '); //Sanitise input filename
	getVoxelSize(dx,dy,dz,um);
	Stack.getDimensions(width,height,channels,slices,frames);
	run("Split Channels");
	
	//Cells counting
	selectWindow('C2-' + title);
	run("Z Project...", "projection=[Sum Slices]");
	run("Convert to Mask");
	run("Smooth");
	run("Convert to Mask");
	run("Erode");
	run("Dilate");
	run("Analyze Particles...", "size=10-Infinity circularity=0.00-0.90 show=Outlines clear summarize");
	close('C2-' + title);
	selectWindow('Drawing of SUM_C2-' + title);
	rename("MAX");
	selectWindow('SUM_C2-' + title);
	rename("MASK");
	//make the composite to compare masks
	run("Combine...", "stack1=MAX stack2=MASK");
	saveAs("PNG", dir + "cells_images/" + out + "_count.png");
	close();
	close('C2-' + title);
	
	//PLA analysis
	selectWindow('C1-' + title);
	run("Z Project...", "projection=[Sum Slices]");
	selectWindow('SUM_C1-' + title);
	// Use the line below for bright datasets (for your samples)
	//setAutoThreshold();
	// Use the line below for Negative Controls
	setAutoThreshold("MaxEntropy dark");
	run("Convert to Mask");
	run("Measure");
	signal = getResult("RawIntDen", 0);
	volume = width*height*slices*dx*dy*dz;
	f = File.append(title + ".tif," + signal + "," + dx + "," + dy + "," + dz + "," + volume, dir + "PLA_data.txt");
	run("Invert LUT");
	rename("mask");
	selectWindow('C1-' + title);
	run("Z Project...", "projection=[Sum Slices]");
	selectWindow('C1-' + title);
	close();
	selectWindow('SUM_C1-' + title);
	rename("original");
	run("8-bit");
	name1="SUM_C1-" + title + "-1";
	name2="SUM_C1-" + title;
	run("Combine...", "stack1=original stack2=mask");
	saveAs("PNG", dir + "PLA_images/" + out + "_analysed.png");
	close();
}
//Saving cells counting as csv file 
if (isOpen("Results")) {
    selectWindow("Results");
    run("Close");
}
selectWindow("Summary");
saveAs("Results", dir + "cells_images/" + "cells_count.csv");
close("cells_count.csv");
